In [50]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import math
import warnings

from IPython.display import display
from sklearn import linear_model
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [51]:
df = pd.read_csv('C:\\Users\\STEVEN\\Documents\\Thinkful\\new_york.csv')
fl = pd.read_csv('C:\\Users\\STEVEN\\Documents\\Thinkful\\florida_crime.csv')

In [52]:
#fill in all missing datas with 0

df = df.fillna(0)
fl = fl.fillna(0)
print(df.loc[0,:])
print(fl.loc[0,:])

City                                      Adams Village
Population                                        1,861
Violent\ncrime                                        0
Murder and\nnonnegligent\nmanslaughter            0.000
Rape\n(revised\ndefinition)1                      0.000
Rape\n(legacy\ndefinition)2                           0
Robbery                                               0
Aggravated\nassault                                   0
Property\ncrime                                      12
Burglary                                              2
Larceny-\ntheft                                      10
Motor\nvehicle\ntheft                                 0
Arson3                                            0.000
Unnamed: 13                                       0.000
Name: 0, dtype: object
City                                      Alachua
Population                                  9,338
Violent\ncrime                                 36
Murder and\nnonnegligent\nmanslaughter     

In [53]:
#NY
df['Population'] = df['Population'].str.replace(',', '')
df = df.fillna(0)
df['Population'] = df['Population'].astype(int)
#FL
fl['Population'] = fl['Population'].str.replace(',', '')
fl = fl.fillna(0)
fl['Population'] = fl['Population'].astype(int)

In [54]:
#Population squared:
df['Population_sq'] = df['Population'].apply(lambda x: x*x)
fl['Population_sq'] = fl['Population'].apply(lambda x: x*x)

In [55]:
#NY Robbery
df['Robbery'] = df['Robbery'].str.replace(',', '')
df = df.fillna(0)
df['Robbery'] = df['Robbery'].astype(int)
df['robbery_cat'] = df['Robbery'].apply(lambda x: 0 if x == 0 else 1)
#FL Robbery
fl['Robbery'] = fl['Robbery'].str.replace(',', '')
fl = fl.fillna(0)
fl['Robbery'] = fl['Robbery'].astype(int)
fl['robbery_cat'] = fl['Robbery'].apply(lambda x: 0 if x == 0 else 1)

In [56]:
#NY Murders
df = df.rename(index=str,columns={'Murder and\nnonnegligent\nmanslaughter': 'Murder'})
df['Murder'] = df['Murder'].astype(int)
df['murder_cat'] = df['Murder'].apply(lambda x: 0 if x == 0 else 1)
#FL Murders
fl = fl.rename(index=str,columns={'Murder and\nnonnegligent\nmanslaughter': 'Murder'})
fl['Murder'] = fl['Murder'].astype(int)
fl['murder_cat'] = fl['Murder'].apply(lambda x: 0 if x == 0 else 1)

In [57]:
#tests
test = df[['Population'] + ['Population_sq'] + ['murder_cat'] + ['robbery_cat']]
test2 = fl[['Population'] + ['Population_sq'] + ['murder_cat'] + ['robbery_cat']]

In [58]:
#Cleaning up property crime: 
df['Property\ncrime'] = df['Property\ncrime'].str.replace(',', '')
df = df.fillna(0)
df['Property\ncrime'] = df['Property\ncrime'].astype(int)

#FL
fl['Property\ncrime'] = fl['Property\ncrime'].str.replace(',', '')
fl = fl.fillna(0)
fl['Property\ncrime'] = fl['Property\ncrime'].astype(int)

In [59]:
#Create regression analysis for new york

regr = linear_model.LinearRegression()
Y = df['Property\ncrime']
X = test
regr.fit(X, Y)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [ 3.46489427e-02 -2.11012230e-09  1.53973792e+01 -9.84856670e+01]

Intercept: 
 -107.22575522520583

R-squared:
0.996123334982826


In [60]:
print(fl.head())

                City  Population Violent\ncrime  Murder  \
0            Alachua        9338             36       0   
1  Altamonte Springs       42040            150       0   
2              Altha         536              1       0   
3       Apalachicola        2269              1       0   
4             Apopka       45397            178       1   

   Rape\n(revised\ndefinition)1  Rape\n(legacy\ndefinition)2  Robbery  \
0                         2.000                        0.000        2   
1                         8.000                        0.000       37   
2                         0.000                        0.000        0   
3                         0.000                        0.000        0   
4                        11.000                        0.000       57   

  Aggravated\nassault  Property\ncrime Burglary Larceny-\ntheft  \
0                  32              166       40             120   
1                 105             1564      300           1,180   
2    

In [61]:
#Investigate new york with crossfolds and holdouts

In [62]:
D = df[['Property\ncrime'] + ['Population'] + ['Population_sq'] + ['murder_cat'] + ['robbery_cat']]
E = fl[['Property\ncrime'] + ['Population'] + ['Population_sq'] + ['murder_cat'] + ['robbery_cat']]
#print(D.corr())
#print(D.head())
Emmett = df[['Property\ncrime'] + ['Population'] + ['Murder'] + ['Robbery']]
#print(Emmett.head())

In [63]:
#print(Emmett.corr())

In [67]:
from sklearn.model_selection import cross_val_score
cross_val_score(regr, X, Y, cv=5)

array([ 8.23865068e-01,  3.85188989e-01,  5.30535820e-01, -2.27699204e+03,
        4.70873532e-01])

In [68]:
from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(regr.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(regr.fit(X, Y).score(X, Y)))

With 20% Holdout: 0.5129367957056183
Testing on Sample: 0.996123334982826


It appears that our data is overfitting to our work.  

In [69]:
print(test.corr())

               Population  Population_sq  murder_cat  robbery_cat
Population          1.000          0.998       0.162        0.065
Population_sq       0.998          1.000       0.133        0.044
murder_cat          0.162          0.133       1.000        0.315
robbery_cat         0.065          0.044       0.315        1.000


In [70]:
print(E.corr())

                 Property\ncrime  Population  Population_sq  murder_cat  \
Property\ncrime            1.000       0.950          0.816       0.396   
Population                 0.950       1.000          0.863       0.384   
Population_sq              0.816       0.863          1.000       0.160   
murder_cat                 0.396       0.384          0.160       1.000   
robbery_cat                0.192       0.201          0.062       0.314   

                 robbery_cat  
Property\ncrime        0.192  
Population             0.201  
Population_sq          0.062  
murder_cat             0.314  
robbery_cat            1.000  


In [71]:
#Create regression analysis for Florida


B = fl['Property\ncrime']
A = test2
regr.fit(A, B)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(A, B))


Coefficients: 
 [ 3.88713592e-02  8.24554662e-10  2.46640506e+02 -6.02647614e+01]

Intercept: 
 -60.973241412406196

R-squared:
0.9029431116923462


In [72]:
#df.sort_values('Population', ascending = False)
#fl.sort_values('Population', ascending = False)

In [73]:
cross_val_score(regr, A, B, cv=10)

array([0.86979429, 0.54962919, 0.87450143, 0.81047541, 0.92325177,
       0.85400705, 0.77544288, 0.42117926, 0.83997227, 0.67038741])

In [74]:
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(regr.fit(A_train, B_train).score(A_test, B_test)))
print('Testing on Sample: ' + str(regr.fit(A, B).score(A, B)))

With 20% Holdout: 0.8040778927387644
Testing on Sample: 0.9029431116923462


The florida sample appears to be holding with a .903 accuracy.  However the New York sample appears all over the place.  
Let's add in a square root of population and burglary categorical and take out population to better determine how our model is functioning.  

In [75]:
#NY & FL Adjustment
#Population rooted:
df['Population_sqrt'] = np.sqrt(df['Population'])
fl['Population_sqrt'] = np.sqrt(fl['Population'])



In [76]:
#Burglary

df['Burglary'] = df['Burglary'].str.replace(',', '')
df = df.fillna(0)
df['Burglary'] = df['Burglary'].astype(int)
df['burglary_cat'] = df['Burglary'].apply(lambda x: 0 if x == 0 else 1)
#FL Robbery
fl['Burglary'] = fl['Burglary'].str.replace(',', '')
fl = fl.fillna(0)
fl['Burglary'] = fl['Burglary'].astype(int)
fl['burglary_cat'] = fl['Burglary'].apply(lambda x: 0 if x == 0 else 1)

In [77]:
#now we create our new models
NYtest2 = df[ ['Population_sqrt'] + ['Population_sq'] + ['burglary_cat'] + ['murder_cat'] + ['robbery_cat']]
FLtest2 = fl[['Population_sqrt'] + ['Population_sq'] + ['burglary_cat'] + ['murder_cat'] + ['robbery_cat']]


In [78]:
# now lets run our tests again.  Beginning with New York

Y = df['Property\ncrime']
X2 = NYtest2
regr.fit(X2, Y)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X2, Y))


Coefficients: 
 [ 1.16945069e+01  1.54515710e-09 -3.45685522e+02  1.58574859e+02
 -2.75380613e+02]

Intercept: 
 -369.3418175575989

R-squared:
0.9926651688007165


In [80]:
cross_val_score(regr, X2, Y, cv=10)

array([ 5.60017829e-01,  4.64124274e-01, -2.41431369e+00,  1.13089177e-02,
        2.29193503e-02,  1.31251462e-01, -5.32322709e+03,  5.88176701e-01,
        7.17464304e-01,  4.65511477e-02])

In [81]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, Y, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(regr.fit(X2_train, y2_train).score(X2_test, y2_test)))
print('Testing on Sample: ' + str(regr.fit(X2, Y).score(X2, Y)))

With 20% Holdout: 0.16641661030773147
Testing on Sample: 0.9926651688007165


In [82]:
# our new correlations
nycor = df[['Property\ncrime'] + ['Population_sqrt'] + ['Population_sq'] + ['burglary_cat'] + ['murder_cat'] + ['robbery_cat']]
flcor = fl[['Property\ncrime'] + ['Population_sqrt'] + ['Population_sq'] + ['burglary_cat'] + ['murder_cat'] + ['robbery_cat']]
print(nycor.corr())

                 Property\ncrime  Population_sqrt  Population_sq  \
Property\ncrime            1.000            0.938          0.991   
Population_sqrt            0.938            1.000          0.903   
Population_sq              0.991            0.903          1.000   
burglary_cat               0.028            0.129          0.015   
murder_cat                 0.191            0.352          0.133   
robbery_cat                0.080            0.257          0.044   

                 burglary_cat  murder_cat  robbery_cat  
Property\ncrime         0.028       0.191        0.080  
Population_sqrt         0.129       0.352        0.257  
Population_sq           0.015       0.133        0.044  
burglary_cat            1.000       0.109        0.306  
murder_cat              0.109       1.000        0.315  
robbery_cat             0.306       0.315        1.000  


In [83]:
print(flcor.corr())

                 Property\ncrime  Population_sqrt  Population_sq  \
Property\ncrime            1.000            0.844          0.816   
Population_sqrt            0.844            1.000          0.585   
Population_sq              0.816            0.585          1.000   
burglary_cat               0.066            0.169          0.021   
murder_cat                 0.396            0.546          0.160   
robbery_cat                0.192            0.387          0.062   

                 burglary_cat  murder_cat  robbery_cat  
Property\ncrime         0.066       0.396        0.192  
Population_sqrt         0.169       0.546        0.387  
Population_sq           0.021       0.160        0.062  
burglary_cat            1.000       0.117        0.333  
murder_cat              0.117       1.000        0.314  
robbery_cat             0.333       0.314        1.000  


In [84]:
#plt.scatter(fl['Property\ncrime'], fl['Population'])
#plt.scatter(fl['Population_sq'], fl['Population_sq2'])
fl.sort_values('Population_sq')

,City,Population,Violent crime,Murder,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Unnamed: 13,Population_sq,robbery_cat,murder_cat,Population_sqrt,burglary_cat
292,3 The FBI determined that the agency's data we...,0,0,0,0.000,0.000,0,0,0,0,0,0,0.000,0.000,0,0,0,0.000,0
290,1 The figures shown in this column for the off...,0,0,0,0.000,0.000,0,0,0,0,0,0,0.000,0.000,0,0,0,0.000,0
291,2 The figures shown in this column for the off...,0,0,0,0.000,0.000,0,0,0,0,0,0,0.000,0.000,0,0,0,0.000,0
101,Indian Creek Village,90,0,0,0.000,0.000,0,0,0,0,0,0,0.000,0.000,8100,0,0,9.487,0
113,Jupiter Inlet Colony,417,0,0,0.000,0.000,0,0,3,0,2,1,0.000,0.000,173889,0,0,20.421,0
147,Manalapan,434,2,0,0.000,0.000,0,2,23,3,20,0,0.000,0.000,188356,0,0,20.833,1
2,Altha,536,1,0,0.000,0.000,0,1,3,3,0,0,0.000,0.000,287296,0,0,23.152,1
156,Melbourne Village,666,0,0,0.000,0.000,0,0,18,1,17,0,0.000,0.000,443556,0,0,25.807,1
33,Cedar Key,696,0,0,0.000,0.000,0,0,21,5,15,1,0.000,0.000,484416,0,0,26.382,1
231,Sea Ranch Lakes,702,1,0,0.000,0.000,0,1,14,4,9,1,0.000,0.000,492804,0,0,26.495,1


In [85]:
#fl['Population_sq2'] = fl['Population'].apply(lambda x: x*x)

In [86]:
B = fl['Property\ncrime']
A2 = FLtest2
regr.fit(A2, B)

print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(A2, B))


Coefficients: 
 [ 1.47153471e+01  3.15164135e-08 -4.53749888e+02  1.49685611e+02
 -4.67214315e+02]

Intercept: 
 -210.22077381233362

R-squared:
0.8758334455982336


In [87]:
 cross_val_score(regr, A2, B, cv=10)       
    

array([0.76925087, 0.573342  , 0.8648246 , 0.11654134, 0.88966214,
       0.78729429, 0.68137594, 0.52840124, 0.84635029, 0.40416198])

In [88]:
A2_train, A2_test, B_train, B_test = train_test_split(A2, B, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(regr.fit(A2_train, B_train).score(A2_test, B_test)))
print('Testing on Sample: ' + str(regr.fit(A2, B).score(A2, B)))

With 20% Holdout: 0.20378405269972122
Testing on Sample: 0.8758334455982336
